In [1]:

import transformers
import datasets
import pandas as pd
import numpy as np

model_checkpoint = "/kaggle/input/huggingfacedebertav3variants/deberta-v3-xsmall"


import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))
        

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/llm-detect-ai-generated-text/sample_submission.csv
/kaggle/input/llm-detect-ai-generated-text/train_prompts.csv
/kaggle/input/llm-detect-ai-generated-text/test_essays.csv
/kaggle/input/llm-detect-ai-generated-text/train_essays.csv
/kaggle/input/huggingfacedebertav3variants/khalidalt-DeBERTa-v3-large/spm.model
/kaggle/input/huggingfacedebertav3variants/khalidalt-DeBERTa-v3-large/config.json
/kaggle/input/huggingfacedebertav3variants/khalidalt-DeBERTa-v3-large/README (1).md
/kaggle/input/huggingfacedebertav3variants/khalidalt-DeBERTa-v3-large/README.md
/kaggle/input/huggingfacedebertav3variants/khalidalt-DeBERTa-v3-large/tokenizer_config.json
/kaggle/input/huggingfacedebertav3variants/khalidalt-DeBERTa-v3-large/tokenizer_config (1).json
/kaggle/input/huggingfacedebertav3variants/khalidalt-DeBERTa-v3-large/pytorch_model.bin
/kaggle/input/huggingfacedebertav3variants/khalidalt-DeBERTa-v3-large/special_tokens_map.json
/kaggle/input/huggingfacedebertav3variants/khalidalt-DeBERT

In [2]:
from datasets import Dataset

#from transformers import AutoModel
#model = AutoModel.from_pretrained("microsoft/deberta-v3-xsmall")

In [3]:
df = pd.read_csv('/kaggle/input/daigt-v2-train-dataset/train_v2_drcat_02.csv')
train = df[df.prompt_name != 'Car-free cities'].reset_index(drop=True)
valid = df[df.prompt_name == 'Car-free cities'].reset_index(drop=True)
train.head()

,text,label,prompt_name,source,RDizzl3_seven
0,Phones\n\nModern humans today are always on th...,0,Phones and driving,persuade_corpus,False
1,This essay will explain if drivers should or s...,0,Phones and driving,persuade_corpus,False
2,Driving while the use of cellular devices\n\nT...,0,Phones and driving,persuade_corpus,False
3,Phones & Driving\n\nDrivers should not be able...,0,Phones and driving,persuade_corpus,False
4,Cell Phone Operation While Driving\n\nThe abil...,0,Phones and driving,persuade_corpus,False


In [4]:
train["source"].value_counts()

source
persuade_corpus                       24037
llama2_chat                            2411
chat_gpt_moth                          2409
mistral7binstruct_v1                   2408
mistral7binstruct_v2                   2406
llama_70b_v1                            984
darragh_claude_v6                       952
darragh_claude_v7                       951
falcon_180b_v1                          899
kingki19_palm                           672
train_essays                            670
cohere-command                          301
palm-text-bison1                        300
radek_500                               250
mistralai/Mistral-7B-Instruct-v0.1      201
NousResearch/Llama-2-7b-chat-hf         200
radekgpt4                               100
Name: count, dtype: int64

In [5]:
ds_train = Dataset.from_pandas(train)
ds_valid = Dataset.from_pandas(valid)
ds_train

Dataset({
    features: ['text', 'label', 'prompt_name', 'source', 'RDizzl3_seven'],
    num_rows: 40151
})

In [6]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained(model_checkpoint, use_fast=True)

/opt/conda/lib/python3.10/site-packages/transformers/convert_slow_tokenizer.py:550: UserWarning: The sentencepiece tokenizer that you are converting to a fast tokenizer uses the byte fallback option which is not implemented in the fast tokenizers. In practice this means that the fast version of the tokenizer can produce unknown tokens whereas the sentencepiece version would have converted these unknown tokens into a sequence of byte tokens matching the original piece of text.
  warnings.warn(


In [7]:
def preprocess_function(examples):
    return tokenizer(examples['text'], max_length=128, padding=True, truncation=True)

In [8]:
ds_train_enc = ds_train.map(preprocess_function, batched=True)

Map:   0%|          | 0/40151 [00:00<?, ? examples/s]

In [9]:
ds_valid_enc = ds_valid.map(preprocess_function, batched=True)

Map:   0%|          | 0/4717 [00:00<?, ? examples/s]

In [10]:
from transformers import AutoModelForSequenceClassification, TrainingArguments, Trainer

num_labels = 2
model = AutoModelForSequenceClassification.from_pretrained(model_checkpoint, num_labels=num_labels)

2024-05-11 10:18:59.076085: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-05-11 10:18:59.076223: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-05-11 10:18:59.233316: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
/opt/conda/lib/python3.10/site-packages/torch/_utils.py:831: UserWarning: TypedStorage is deprecated. It will be removed in the future and UntypedStorage will be the only storage class. This should only matter to you if you are using storages directly.  To access UntypedStorage directly, use tensor.untyped_storage() instead of tensor.storage()
  return self.fget.

In [11]:
metric_name = "roc_auc"
model_name = "deberta-xsmall"
train_batch_size = 4
eval_batch_size = 32
grad_acc = 4

In [12]:
num_steps = len(train) // (train_batch_size * grad_acc)
num_steps

2509

In [13]:
args = TrainingArguments(
    f"{model_name}-finetuned",
    evaluation_strategy = "steps",
    save_strategy = "steps",
    eval_steps = num_steps // 3,
    save_steps = num_steps // 3,
    learning_rate=2e-5,
    per_device_train_batch_size=train_batch_size,
    per_device_eval_batch_size=eval_batch_size,
    gradient_accumulation_steps=grad_acc,
    num_train_epochs=1,
    weight_decay=0.01,
    load_best_model_at_end=False,
    metric_for_best_model=metric_name,
    report_to='none', # change to wandb after enabling internet access
)

In [14]:
from sklearn.metrics import roc_auc_score

def compute_metrics(eval_pred):
    logits, labels = eval_pred
    probs = np.exp(logits) / np.sum(np.exp(logits), axis=-1, keepdims=True)
    auc = roc_auc_score(labels, probs[:,1], multi_class='ovr')
    return {"roc_auc": auc}

In [15]:
trainer = Trainer(
    model,
    args,
    train_dataset=ds_train_enc,
    eval_dataset=ds_valid_enc,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics
)

/opt/conda/lib/python3.10/site-packages/accelerate/accelerator.py:436: FutureWarning: Passing the following arguments to `Accelerator` is deprecated and will be removed in version 1.0 of Accelerate: dict_keys(['dispatch_batches', 'split_batches', 'even_batches', 'use_seedable_sampler']). Please pass an `accelerate.DataLoaderConfiguration` instead: 
dataloader_config = DataLoaderConfiguration(dispatch_batches=None, split_batches=False, even_batches=True, use_seedable_sampler=True)
  warnings.warn(


In [16]:
trainer.train()

/opt/conda/lib/python3.10/site-packages/torch/nn/parallel/_functions.py:68: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn('Was asked to gather along dimension 0, but all '


Step,Training Loss,Validation Loss,Roc Auc
836,0.113400,0.105583,0.999544


/opt/conda/lib/python3.10/site-packages/torch/nn/parallel/_functions.py:68: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn('Was asked to gather along dimension 0, but all '


TrainOutput(global_step=1254, training_loss=0.06341442575104879, metrics={'train_runtime': 916.6882, 'train_samples_per_second': 43.8, 'train_steps_per_second': 1.368, 'total_flos': 660858896941056.0, 'train_loss': 0.06341442575104879, 'epoch': 1.0})

In [17]:
test = pd.read_csv('/kaggle/input/llm-detect-ai-generated-text/test_essays.csv')
test_ds = Dataset.from_pandas(test)
test_ds_enc = test_ds.map(preprocess_function, batched=True)

Map:   0%|          | 0/3 [00:00<?, ? examples/s]

In [18]:
test_preds = trainer.predict(test_ds_enc)

In [19]:
logits = test_preds.predictions
probs = np.exp(logits) / np.sum(np.exp(logits), axis=-1, keepdims=True)
sub = pd.DataFrame()
sub['id'] = test['id']
sub['generated'] = probs[:,1]
sub.to_csv('submission.csv', index=False)
sub.head()

,id,generated
0,0000aaaa,0.030469
1,1111bbbb,0.991596
2,2222cccc,0.957312


In [20]:
res = []
for src in valid.source.unique():
    if src in ['train_essays', 'persuade_corpus', 'original_moth']: continue
    test2  = valid[valid['source'].isin([src, 'train_essays'])]
    test_ds2 = Dataset.from_pandas(test2)
    test_ds_enc2 = test_ds2.map(preprocess_function, batched=True)
    eval_result = trainer.evaluate(test_ds_enc2)
    score = eval_result['eval_roc_auc']
    res.append(f'{src}: {score}')
    
for r in res: print(r)

Map:   0%|          | 0/723 [00:00<?, ? examples/s]

/opt/conda/lib/python3.10/site-packages/torch/nn/parallel/_functions.py:68: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn('Was asked to gather along dimension 0, but all '


Map:   0%|          | 0/896 [00:00<?, ? examples/s]

/opt/conda/lib/python3.10/site-packages/torch/nn/parallel/_functions.py:68: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn('Was asked to gather along dimension 0, but all '


Map:   0%|          | 0/757 [00:00<?, ? examples/s]

...

Map:   0%|          | 0/808 [00:00<?, ? examples/s]

/opt/conda/lib/python3.10/site-packages/torch/nn/parallel/_functions.py:68: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn('Was asked to gather along dimension 0, but all '


Map:   0%|          | 0/1420 [00:00<?, ? examples/s]

/opt/conda/lib/python3.10/site-packages/torch/nn/parallel/_functions.py:68: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn('Was asked to gather along dimension 0, but all '


mistral7binstruct_v2: 0.9736562942008486
llama_70b_v1: 0.9957716860121386
darragh_claude_v7: 0.9915700141442716
radek_500: 0.998320719949058
darragh_claude_v6: 0.9913979736166036
falcon_180b_v1: 0.9963783457508626
mistral7binstruct_v1: 0.9698929076581128
chat_gpt_moth: 0.9675769774779677
llama2_chat: 0.9609103767519609
NousResearch/Llama-2-7b-chat-hf: 0.9970233697143702
mistralai/Mistral-7B-Instruct-v0.1: 0.997871287128713
palm-text-bison1: 0.9915700141442716
cohere-command: 0.9912588401697313
radekgpt4: 0.9952105535871834
kingki19_palm: 0.9994088368965127


In [22]:
import shutil
shutil.make_archive('deberta_fine_tuned', 'zip', '/kaggle/working/deberta-xsmall-finetuned')

'/kaggle/working/deberta_fine_tuned.zip'